# Load a model and conduct a simulation

# Purpose
It is now possible to load simulation model from disk. This notebook explore this functionality.

# Methodology
* The simulation model have been stored to disk.
* Load one model and simulate one model test.

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm 
from vessel_manoeuvring_models.models.vmm import ModelSimulator


In [ ]:
from vessel_manoeuvring_models.to_mlflow import mlflow
import multiprocessing

In [ ]:
from vessel_manoeuvring_models.data.mdl import load_test

## Parameters

In [ ]:
run_params = {
    'experiment' : 'VCT_linear',
    'model_test_id' : 22770,
    'model_test_dir_path' : os.path.abspath('../../../data/processed/kalman'),
    'model' : os.path.abspath('../../../models/model_VCT_linear.pkl'),
    'run_name' : '22770',
}

## Run

In [ ]:
artifact_dir = 'artifacts'
if not os.path.exists(artifact_dir):
    os.mkdir(artifact_dir)

mlflow.set_experiment(run_params['experiment'])

with mlflow.start_run(run_name=run_params['run_name']) as run:
    
    ## Meta data
    log_params = run_params.copy()
    log_params.pop('experiment')
    log_params.pop('run_name')
    mlflow.log_params(run_params)
        
    ## Load and resimulate
    df, meta_data = load_test(**run_params)
    mlflow.log_params(meta_data.dropna())
    
    model = ModelSimulator.load(run_params['model'])
    result = model.simulate(df)
    
    ## Result to MLFlow
    result.to_mlflow()
    